# Model Extraction Notebook
---

## Import Depedencies

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Google Mouting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/capstone-project/asl-signs

Mounted at /content/drive
/content/drive/MyDrive/capstone-project/asl-signs


## Function Helper

In [7]:
ROWS_PER_FRAME = 543 # Number of landmark each frame
SEQUENCE_LENGTH = 30 # 30 frame
NUM_LABELS = 250

lipsUpperOuter = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter = [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
lipsUpperInner = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner = [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]

# Landmark order each frame: IDX:0 | Lips -> Left Hand -> Pose -> Right Hand | IDX:543
lipsIDX = np.array(lipsUpperOuter + lipsLowerOuter + lipsUpperInner + lipsLowerInner)
lhIDX = 468
poseIDX = 489
rhIDX = 522

* Landmark Extractor

In [9]:
def extract_keypoint(pq_path):
    data = pd.read_parquet(pq_path, columns=['x', 'y', 'z'])
    data.replace(np.nan, 0, inplace=True) # Imputing empty left/right hand landmark
    total_frame = int(len(data) / ROWS_PER_FRAME)
    
    landmarks = []
    sequences_interval = 1 if total_frame < SEQUENCE_LENGTH else total_frame // SEQUENCE_LENGTH
    
    for frame in range(0, SEQUENCE_LENGTH*sequences_interval, sequences_interval):
      try:
        boundary = ROWS_PER_FRAME * frame

        lips = np.array(data.iloc[lipsIDX + boundary]).flatten()
        lh = np.array(data.iloc[lhIDX+boundary : poseIDX+boundary]).flatten()
        pose = np.array(data.iloc[poseIDX+boundary : rhIDX+boundary]).flatten()
        rh = np.array(data.iloc[rhIDX+boundary : ROWS_PER_FRAME*(frame+1)]).flatten()

        landmarks.append(np.concatenate([lips,lh,pose,rh]))
      except:
        landmarks.append(np.zeros(354)) # Imputing empty frame

    return np.array(landmarks)

* One Hot Decoder

In [2]:
def oneHot_decoder(oneHot_label):
    return np.array([np.where(encodedLabel == 1)[0][0] for encodedLabel in oneHot_label])

* Prediction Result Decoder

In [3]:
def prediction_decoder(prediction):
    return np.array([np.argmax(probability) for probability in prediction])

## Representative Dataset for Data Generator

Take 10% of data tarining

In [4]:
x_train_path = np.load('x_train_path.npy', allow_pickle=True)
x_train_path.shape

(75581,)

In [5]:
y_train = np.load('y_train.npy')
y_train.shape

(75581, 250)

In [6]:
x_tmp, x_repData_path, y_tmp, y_repData = train_test_split(x_train_path, y_train, test_size=0.1, random_state=42, stratify=y_train)
del x_tmp, y_tmp
del x_train_path, y_train

In [10]:
x_repData = []
size = len(x_repData_path)

for i,path in enumerate(x_repData_path):
  print(f"[{i+1}/{size}] {path}")
  x_repData.append(extract_keypoint(path))

[1/7559] train_landmark_files/37779/1803092142.parquet
[2/7559] train_landmark_files/61333/701035673.parquet
[3/7559] train_landmark_files/26734/3054847588.parquet
[4/7559] train_landmark_files/16069/548311639.parquet
[5/7559] train_landmark_files/16069/4096212145.parquet
[6/7559] train_landmark_files/29302/1228300982.parquet
[7/7559] train_landmark_files/22343/1525593586.parquet
[8/7559] train_landmark_files/28656/2233471654.parquet
[9/7559] train_landmark_files/22343/1530193497.parquet
[10/7559] train_landmark_files/28656/2634539219.parquet
[11/7559] train_landmark_files/30680/1363776467.parquet
[12/7559] train_landmark_files/28656/1788485595.parquet
[13/7559] train_landmark_files/55372/3894347036.parquet
[14/7559] train_landmark_files/26734/1612935424.parquet
[15/7559] train_landmark_files/37779/3274972616.parquet
[16/7559] train_landmark_files/4718/3209183075.parquet
[17/7559] train_landmark_files/29302/2523098902.parquet
[18/7559] train_landmark_files/55372/4263517894.parquet
[19/

In [13]:
print(np.array(x_repData).shape)
print(y_repData.shape)

(7559, 30, 354)
(7559, 250)


In [14]:
np.save('x_repData', x_repData)
np.save('y_repData', y_repData)

## Convert Model to TFLite

### Load TFLite Converter and Representative Dataset

In [8]:
keras_model = tf.keras.models.load_model('saved_models/v3.1.2.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

In [83]:
x_repData = np.load('x_repData.npy').astype(np.float32)

In [84]:
y_repData = np.load('y_repData.npy')

### Optimization

* Post-Training Quantization

In [9]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

* Post-Training Integer Quantization

In [27]:
# def representative_data_gen():
#     for sample in x_repData:
#         yield [sample]

In [28]:
# converter.representative_dataset = representative_data_gen

* Full Integer Quantization

In [6]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

### Convert The Model

In [12]:
converter._experimental_lower_tensor_list_ops = False
# converter.experimental_new_converter = True

In [ ]:
tflite_model = converter.convert()
tflite_model_file = 'saved_models/wasl_recognition_v3.1.3.tflite'

In [17]:
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

## Convert Model to TFLite [Fusion Codelab Reference]

In [4]:
keras_model = tf.keras.models.load_model('saved_models/v3.1.2.h5')

In [5]:
run_model = tf.function(lambda x: keras_model(x))
# This is important, let's fix the input size.
BATCH_SIZE = 1
STEPS = 30
INPUT_SIZE = 354
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], keras_model.inputs[0].dtype))

# model directory.
MODEL_DIR = "saved_models/v3.1.2-FusionLab"
keras_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
INFO:tensorflow:Assets written to: saved_models/v3.1.2-FusionLab\assets


In [6]:
tflite_model = converter.convert()
tflite_model_file = 'saved_models/wasl_recognition_v3.1.2_fusion.tflite'

In [7]:
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

## Test TFLite Model

In [8]:
x_tmp, x_testTFLite, y_tmp, y_testTFLite = train_test_split(np.load('x_test.npy'), np.load('y_test.npy'), test_size=0.1)
del x_tmp, y_tmp

In [9]:
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [10]:
predictions = []

test_labels, test_data = [], []
for data, label in tqdm(zip(x_testTFLite.astype('float32'),y_testTFLite)):
    interpreter.set_tensor(input_index, np.expand_dims(data, axis=0))
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    
    test_labels.append(label)
    test_data.append(data)

945it [00:02, 372.44it/s]


In [11]:
accuracy_score(oneHot_decoder(test_labels), prediction_decoder(predictions))

0.7185185185185186